In [1]:
import numpy as np
import cv2
import apriltag

# Loading the calibration data (K matrix and distortion coefficients)
calibration_data = np.load('calibration_set2.npz')
K = calibration_data['camMatrix']  # Intrinsic camera matrix
dist_coeffs = calibration_data['distCoeff']  # Distortion coefficients

# Loading the image where the AprilTag is located
image = cv2.imread('/Users/devikakodi/Desktop/q4/4.JPG')
# Resize the image to reduce memory usage
image = cv2.resize(image, (int(image.shape[1] / 2), int(image.shape[0] / 2)))


# Checking if the image was loaded successfully
if image is None:
    print("Error: Could not load image.")
else:
    print("Image loaded successfully!")

    # Converting the image to grayscale for the AprilTag detector
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initializing the AprilTag detector with the tag family 'tag25h9'
    options = apriltag.DetectorOptions(families='tag25h9')
    detector = apriltag.Detector(options)

    # Detecting AprilTags in the image
    tags = detector.detect(gray_image)


Image loaded successfully!


In [2]:
# Checking if any tags are detected
if len(tags) > 0:
        for tag in tags:
            # Getting the detected AprilTag ID
            tag_id = tag.tag_id
            print(f"Detected AprilTag ID: {tag_id}")

            # Getting the 2D image points of the AprilTag corners (4 corners)
            image_points = tag.corners.reshape(4, 2)

            # Defining the 3D object points of the AprilTag's corners in the tag's coordinate system
            tag_size = 0.05  # Setting the real-world size of the tag in meters from online source
            object_points = np.array([
                [0, 0, 0],
                [tag_size, 0, 0],
                [tag_size, tag_size, 0],
                [0, tag_size, 0]
            ], dtype=np.float32)

            # Estimating the pose of the AprilTag using solvePnP
            retval, rvec, tvec = cv2.solvePnP(object_points, image_points, K, dist_coeffs)


Detected AprilTag ID: 1


In [ ]:
if retval:
                # Defining 3D points for a cube based on the AprilTag's position
                cube_size = tag_size
                cube_points = np.array([
                    [0, 0, 0], [cube_size, 0, 0], [cube_size, cube_size, 0], [0, cube_size, 0],  # Base of the cube
                    [0, 0, -cube_size], [cube_size, 0, -cube_size], [cube_size, cube_size, -cube_size], [0, cube_size, -cube_size]  # Top of the cube
                ], dtype=np.float32)

                # Projecting the cube's 3D points onto the 2D image plane
                projected_points, _ = cv2.projectPoints(cube_points, rvec, tvec, K, dist_coeffs)
                
                # Drawing the cube on the image
                projected_points = projected_points.reshape(-1, 2).astype(int)
                
                # Function for drawing the 3D cube
                def draw_cube(img, pts):
                    # Drawing the base of the cube
                    cv2.drawContours(img, [pts[:4]], -1, (0, 255, 0), 2)
                    # Drawing the pillars of the cube
                    for i in range(4):
                        cv2.line(img, tuple(pts[i]), tuple(pts[i + 4]), (255, 0, 0), 2)
                    # Drawing the top of the cube
                    cv2.drawContours(img, [pts[4:]], -1, (0, 0, 255), 2)

                # Drawing the projected cube on the image
                draw_cube(image, projected_points)

                # Displaying the image with the cube overlaid on the AprilTag
                cv2.imshow('Augmented Reality Cube on AprilTag', image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

else:
                print("Pose estimation failed.")